In [45]:
import time
import datetime
import re
from itertools import product
from collections import OrderedDict
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pymongo
from pymongo import MongoClient
from pprint import pprint

In [17]:
def writeLog(log_text, function_name):
    with open('web_parcer.log', 'a') as file:
        file.write(str(datetime.datetime.now()) + ":\t" + function_name + ': \t' + log_text + '\n')


def goToPage(query, driver):
    start_loading = datetime.datetime.now()
    try:
        driver.set_page_load_timeout(120)
        driver.get("https://www.aviasales.ru/search/{}".format(query))
    except WebDriverException:
        writeLog('Page loading timeout :' + query, 'goToPage')
    
    try:
        time.sleep(3)    
        wait_seconds = 600
        wait = WebDriverWait(driver, wait_seconds)
        wait.until(EC.invisibility_of_element((By.CLASS_NAME, "countdown")))
        loading_duration = (datetime.datetime.now() - start_loading).seconds
        printDelimiter('*')
        print("$$$$$   page loaded for {} seconds   $$$$$".format(loading_duration))
        time.sleep(1)
    except:
        print("\x1b[31m\"CAN'T WAIT THE PAGE LOAD ({} seconds)\"\x1b[0m".format(wait_seconds))
    

def closeExtraTab(driver): # В предположении что драйвер находится на рекламном окне после его открытия
    num_of_tabs = len(driver.window_handles)
    if num_of_tabs == 2:
        main_window_handle = None
        if driver.current_window_handle == driver.window_handles[0]:
            driver.switch_to_window(driver.window_handles[1])
            main_window_handle = driver.window_handles[0]
        elif driver.current_window_handle == driver.window_handles[1]:
            driver.switch_to_window(driver.window_handles[0])
            main_window_handle = driver.window_handles[1]
        title_of_closed_tab = driver.title
        driver.close()
        driver.switch_to_window(main_window_handle)
        writeLog("Extra tab *{}* has been closed".format(title_of_closed_tab), 'closeExtraTab')
    
    
def printDelimiter(char):
    delimiter = ''
    for i in range(118):
        delimiter += char
    print(delimiter)
    
    
def getCollection():
    client = MongoClient()
    db = client.aviasales_database # getting a database
    return db.ticket_collection_v2

In [36]:
#hardcoded_year = None################

def processPage(driver, collection):
    tickets = driver.find_elements_by_class_name("ticket")
    tickets = removeAdvertisingTickets(tickets)
    tickets = removeAbsentTickets(tickets)
    print(driver.title[-9:], '\n')
    for ticket in tickets:
        collection.insert_one(processTicket(ticket))
        closeExtraTab(driver)
        
        
def removeAdvertisingTickets(tickets):
    for ticket in tickets:
        ticket_opener_text = ticket.find_element_by_class_name("ticket__expand-button").text
        if ticket_opener_text == 'Реклама':
            tickets.remove(ticket)
    return tickets


def removeAbsentTickets(tickets):
    for ticket in tickets:
        ticket_price = (ticket.find_element_by_class_name("buy-button__price").text).replace('\u2009', '')
        if ticket_price == '':
            tickets.remove(ticket)
    return tickets
        
        
def processTicket(ticket):
    ticket_price = int((ticket.find_element_by_class_name("buy-button__price-num").text).replace('\u2009', ''))
    
    ticket_opener = ticket.find_element_by_class_name("ticket__expand-button")
    ticket_opener.click()
    
    ticket_content = ticket.find_element_by_class_name("ticket__content")
    
    flights = ticket_content.find_elements_by_class_name("ticket-segment")
    
    print("crawl_date_time: ", datetime.datetime.now())                    # debug
    print("price: ", ticket_price)                                         # debug
    
    post = dict()
    post['crawl_date_time'] = datetime.datetime.now()
    post['price'] = ticket_price
    post['flights'] = list()
    
    for flight in flights:
        post['flights'].append(processFlight(post, flight))
    
    return post

        
def processFlight(post, flight):
    flight_info = dict()
    flight_info['details'] = list()
    
    general_flight_info = getGeneralFlightInfo(flight)
    flight_info['city'] = general_flight_info[0]
    flight_info['date_time'] = general_flight_info[1]
    flight_info['total_time'] = general_flight_info[2]
    
    #nonlocal hardcoded_year
    #hardcoded_year = flight_general_info['date_time']['from'].year#############################
    
    
    detailed_flight_info = getDetailedFlightInfo(flight)
    for info in detailed_flight_info:
        flight_info['details'].append(info)
        
    flight_info_order = ['total_time', 'date_time', 'city']        # debug
    for info in flight_info_order:                                 # debug
        print('\t' + info + ": ", flight_info[info])               # debug
        
    print('\n')

    return flight_info
    
    
def getGeneralFlightInfo(flight):
# Общая информация
    flight_from = getGeneralPointInfo(flight, "segment-route__endpoint.origin")

    flight_info_path = getGeneralPathInfo(flight)
    
    flight_to = getGeneralPointInfo(flight, "segment-route__endpoint.destination")
    
    flight_info_city = {'from' : flight_from[0],
                        'to' : flight_to[0]}
    flight_info_date_time = {'from' : flight_from[1],
                             'to' : flight_to[1]}
    
    return flight_info_city, flight_info_date_time, flight_info_path


def getGeneralPathInfo(flight):
    flight_path = flight.find_element_by_class_name("segment-route__route_wrap")
    flight_total_time = re.split(' ', flight_path.find_element_by_class_name("segment-route__duration").text)
    flight_total_time_info = getHoursAndMinutes(flight_total_time)
    return flight_total_time_info
    
    
def getHoursAndMinutes(time_string):
    hours = 0
    minutes = 0
    days = 0
    for word in time_string:
        if word[-1] == 'ч':
            hours = word[:-1]
        elif word[-1] == 'м':
            minutes = word[:-1]
        elif word[-1] == 'д':
            days = word[:-1]
    return {'days' : days, 'hours' : hours, 'minutes' : minutes}
    
    
def getGeneralPointInfo(flight, segment_route):
    flight_point = flight.find_element_by_class_name(segment_route)
    flight_city = flight_point.find_element_by_class_name("segment-route__city").text
    time_point = re.split(':', flight_point.find_element_by_class_name("segment-route__time").text)
    date_point = re.split(' ', flight_point.find_element_by_class_name("segment-route__date").text)
    flight_date_time_point = datetime.datetime(int(date_point[2][:-1]), 
                                               translateDate(date_point[1]), 
                                               int(date_point[0]), 
                                               int(time_point[0]), 
                                               int(time_point[1]))
    return flight_city, flight_date_time_point

    
def translateDate(month):
    date_translator = {'янв' : 1, 'фев' : 2, 'мар' : 3, 'апр' : 4 ,'май' : 5 ,'июн' : 6, 'июл' : 7, 'авг' : 8,
                       'сен' : 9, 'окт' : 10, 'ноя' : 11, 'дек' : 12}
    return date_translator[month]
    
    
def getDetailedFlightInfo(flight):
    flight_info = list()
    try:
    # Детали перелета
        time.sleep(1)
    
        flight_itinerary = flight.find_element_by_class_name("ticket-segment__details")

        # Flight legs
        flight_legs = flight_itinerary.find_elements_by_class_name("ticket-segment__flight")
        flight_legs_info = list()

        for flight_leg in flight_legs:
            flight_legs_info.append(getDetailedFlightLegInfo(flight_leg))


        # Flight stops    
        flight_stops = flight_itinerary.find_elements_by_class_name("ticket-segment__stop")
        flight_stops_info = list()

        for flight_stop in flight_stops:
            flight_stops_info.append(getDetailedFlightStopInfo(flight_stop))


        # Миксую через один инфу по остановкам и инфу по пересадкам (для восстановления хронологии в выводе)
        flight_info = mixFlightInfo(flight_legs_info, flight_stops_info)

        #for info in flight_info:                                               # debug
        #    if info['type'] == 'leg':                                          # debug
        #        print("\n\tflight date time: ", info['date_time'],             # debug
        #              "\n\t\tairport: ", info['airport'],                      # debug
        #              "\n\t\tairline: ", info['airline'],                      # debug
        #              "\n\t\tflight number: ", info['flight_number'])          # debug
        #    elif info['type'] == 'stop':                                       # debug
        #        print("\n\tstop time: ", info['time'],                         # debug
        #              "\n\tstop place: ", info['place'])                       # debug

        #print("\n")                                                        # debug
    except NoSuchElementException as exception:
        #print("\x1b[31m\"red\"\x1b[0m")
        print("\x1b[31m\"Element not found (getDetailedFlightInfo): ", exception, "\"\x1b[0m")
    return flight_info 
    
                                 
def mixFlightInfo(flight_legs_info, flight_stops_info):
    flight_info = [flight_stops_info[i//2] 
                   if i%2 else flight_legs_info[i//2] 
                   for i in range(len(flight_stops_info)*2+1)]
    return flight_info
                                 
    
def getDetailedFlightStopInfo(flight_stop):
    flight_attrs_tmp = dict()
    flight_attrs_tmp['place'] = flight_stop.find_element_by_class_name("ticket-stop__place").text
    flight_stop_time = re.split(' ', flight_stop.find_element_by_class_name("ticket-stop__time").text)
    flight_attrs_tmp['time'] = getHoursAndMinutes(flight_stop_time)
    flight_attrs_tmp['type'] = 'stop'
    return flight_attrs_tmp
    

def getDetailedFlightLegInfo(flight_leg):
    try:
        flight_attrs_tmp = dict()
        
        flight_attrs_tmp['type'] = 'leg'

        flight_from_info = flight_leg.find_elements_by_class_name("ticket-flight__route-info")[0]
        flight_to_info = flight_leg.find_elements_by_class_name("ticket-flight__route-info")[1]

        flight_date_time_from = getDetailedFlightLegDatetimeInfo(flight_from_info)
        flight_date_time_to = getDetailedFlightLegDatetimeInfo(flight_to_info)
        
        flight_attrs_tmp['date_time'] = {'from' : flight_date_time_from,
                                         'to' : flight_date_time_to}

        flight_attrs_tmp['airport'] = getDetailedFlightLegAirportInfo(flight_from_info, flight_to_info)

        flight_airline = flight_leg.find_element_by_class_name("ticket-carrier")
        flight_attrs_tmp['airline'] = flight_airline.find_element_by_css_selector('img').get_attribute('alt')
        
        flight_number = flight_leg.find_element_by_class_name("ticket-flight__title")
        flight_attrs_tmp['flight_number'] = flight_number.text[5:]
    
    except NoSuchElementException as exception:
        print("\x1b[31m\"Element not found (getDetailedFlightLegInfo): ", exception, "\"\x1b[0m")
    return flight_attrs_tmp


def getDetailedFlightLegAirportInfo(flight_from_info, flight_to_info):
    flight_airport_name_from = flight_from_info.find_element_by_class_name("ticket-flight__name").text
    flight_airport_iata_from = flight_from_info.find_element_by_class_name("ticket-flight__iata").text
    flight_airport_name_to = flight_to_info.find_element_by_class_name("ticket-flight__name").text
    flight_airport_iata_to = flight_to_info.find_element_by_class_name("ticket-flight__iata").text
    flight_airport_info = {'from' : 
                                   {'name' : flight_airport_name_from,
                                    'iata' : flight_airport_iata_from},
                           'to' : 
                                   {'name' : flight_airport_name_to,
                                    'iata' : flight_airport_iata_to}
                          }
    return flight_airport_info


def getDetailedFlightLegDatetimeInfo(flight_info):
    
    flight_time = re.split(':', flight_info.find_element_by_class_name("ticket-flight__departure-time").text)
    flight_date = re.split(' ', flight_info.find_element_by_class_name("ticket-flight__departure-date").text)
    
    #ATTENTION: создаю багу того что год в дате проставляется как год даты начала поездки
    flight_date_time= datetime.datetime(2019, #######hardcode######################################
                                        translateDate(flight_date[1][:-1]),
                                        int(flight_date[0]),
                                        int(flight_time[0]),
                                        int(flight_time[1])) 
    return flight_date_time


In [34]:
driver = webdriver.Firefox()
driver.get('https://www.aviasales.ru/search/MOW1307STO1')

In [50]:
def crawlByList(destinatoins_dates_pairs, driver): 
    collection = getCollection()
    query = 'MOW'
    for pair in dastinations_dates_pairs:
        query = 'MOW' + pair[1] + pair[0] + '1'
        goToPage(query, driver)
        processPage(driver, collection)
        destinations_dates_pairs.remove(pair)

#При остановке паука можно продолжить поиск с той пары на которой все сломалось
def makeDateDestinationPairs(destinations, dates): # dates - что то в духе ['1304','1404', ...]
    return list(product(destinations, dates))

In [ ]:
# НЕ ЗАПУСКАТЬ ПРИ ПОЛОМКЕ (создаст заново все пары городов и дат)
city_time_pairs = makeDateDestinationPairs(city_list, time_list_july) 

In [37]:
crawlByList(city_time_pairs, driver)

**********************************************************************************************************************
$$$$$   page loaded for 25 seconds   $$$$$
MOW → ALC 

crawl_date_time:  2019-04-17 21:14:32.643226
price:  10699
	total_time:  {'minutes': '30', 'hours': '17', 'days': 0}
	date_time:  {'from': datetime.datetime(2019, 7, 1, 17, 0), 'to': datetime.datetime(2019, 7, 2, 9, 30)}
	city:  {'from': 'Москва', 'to': 'Аликанте'}


crawl_date_time:  2019-04-17 21:14:34.565416
price:  11753
	total_time:  {'minutes': '45', 'hours': '8', 'days': 0}
	date_time:  {'from': datetime.datetime(2019, 7, 1, 9, 30), 'to': datetime.datetime(2019, 7, 1, 17, 15)}
	city:  {'from': 'Москва', 'to': 'Аликанте'}


crawl_date_time:  2019-04-17 21:14:36.534038
price:  11877
	total_time:  {'minutes': '50', 'hours': '4', 'days': '1'}
	date_time:  {'from': datetime.datetime(2019, 7, 1, 5, 40), 'to': datetime.datetime(2019, 7, 2, 9, 30)}
	city:  {'from': 'Москва', 'to': 'Аликанте'}


crawl_date_time:  201

crawl_date_time:  2019-04-17 21:16:18.686541
price:  12435
	total_time:  {'minutes': '30', 'hours': '23', 'days': 0}
	date_time:  {'from': datetime.datetime(2019, 7, 1, 17, 0), 'to': datetime.datetime(2019, 7, 2, 15, 30)}
	city:  {'from': 'Москва', 'to': 'Антверпен'}


**********************************************************************************************************************
$$$$$   page loaded for 24 seconds   $$$$$
MOW → BCN 

crawl_date_time:  2019-04-17 21:16:47.045394
price:  8236
	total_time:  {'minutes': '45', 'hours': '24', 'days': 0}
	date_time:  {'from': datetime.datetime(2019, 7, 1, 22, 10), 'to': datetime.datetime(2019, 7, 2, 21, 55)}
	city:  {'from': 'Москва', 'to': 'Барселона'}


crawl_date_time:  2019-04-17 21:16:49.550542
price:  9250
	total_time:  {'minutes': '20', 'hours': '5', 'days': '1'}
	date_time:  {'from': datetime.datetime(2019, 7, 1, 16, 15), 'to': datetime.datetime(2019, 7, 2, 20, 35)}
	city:  {'from': 'Москва', 'to': 'Барселона'}


crawl_date_time

crawl_date_time:  2019-04-17 21:18:47.804100
price:  7175
	total_time:  {'minutes': '55', 'hours': '2', 'days': 0}
	date_time:  {'from': datetime.datetime(2019, 7, 1, 12, 40), 'to': datetime.datetime(2019, 7, 1, 14, 35)}
	city:  {'from': 'Москва', 'to': 'Берлин'}


crawl_date_time:  2019-04-17 21:18:49.674656
price:  7590
	total_time:  {'minutes': '20', 'hours': '4', 'days': 0}
	date_time:  {'from': datetime.datetime(2019, 7, 1, 4, 50), 'to': datetime.datetime(2019, 7, 1, 8, 10)}
	city:  {'from': 'Москва', 'to': 'Берлин'}


**********************************************************************************************************************
$$$$$   page loaded for 21 seconds   $$$$$
MOW → BRN 

**********************************************************************************************************************
$$$$$   page loaded for 20 seconds   $$$$$
MOW → BLQ 

crawl_date_time:  2019-04-17 21:19:36.830149
price:  7440
	total_time:  {'minutes': '5', 'hours': '15', 'days': 0}
	date_t

	total_time:  {'minutes': '25', 'hours': '9', 'days': '1'}
	date_time:  {'from': datetime.datetime(2019, 7, 1, 17, 5), 'to': datetime.datetime(2019, 7, 3, 1, 30)}
	city:  {'from': 'Москва', 'to': 'Брно'}


crawl_date_time:  2019-04-17 21:21:25.437672
price:  38842
	total_time:  {'minutes': '35', 'hours': '7', 'days': 0}
	date_time:  {'from': datetime.datetime(2019, 7, 1, 11, 55), 'to': datetime.datetime(2019, 7, 1, 18, 30)}
	city:  {'from': 'Москва', 'to': 'Брно'}


crawl_date_time:  2019-04-17 21:21:27.760970
price:  38918
	total_time:  {'minutes': '35', 'hours': '20', 'days': 0}
	date_time:  {'from': datetime.datetime(2019, 7, 1, 4, 20), 'to': datetime.datetime(2019, 7, 1, 23, 55)}
	city:  {'from': 'Москва', 'to': 'Брно'}


crawl_date_time:  2019-04-17 21:21:30.456301
price:  42035
	total_time:  {'minutes': '25', 'hours': '11', 'days': 0}
	date_time:  {'from': datetime.datetime(2019, 7, 1, 8, 5), 'to': datetime.datetime(2019, 7, 1, 18, 30)}
	city:  {'from': 'Москва', 'to': 'Брно'}




crawl_date_time:  2019-04-17 21:24:12.627739
price:  13571
	total_time:  {'minutes': '50', 'hours': '2', 'days': 0}
	date_time:  {'from': datetime.datetime(2019, 7, 1, 10, 10), 'to': datetime.datetime(2019, 7, 1, 12, 0)}
	city:  {'from': 'Москва', 'to': 'Будапешт'}


**********************************************************************************************************************
$$$$$   page loaded for 24 seconds   $$$$$
MOW → VLC 

crawl_date_time:  2019-04-17 21:24:40.645901
price:  12854
	total_time:  {'minutes': '45', 'hours': '2', 'days': '1'}
	date_time:  {'from': datetime.datetime(2019, 7, 1, 20, 40), 'to': datetime.datetime(2019, 7, 2, 22, 25)}
	city:  {'from': 'Москва', 'to': 'Валенсия'}


crawl_date_time:  2019-04-17 21:24:42.830092
price:  12854
	total_time:  {'minutes': 0, 'hours': '8', 'days': '1'}
	date_time:  {'from': datetime.datetime(2019, 7, 1, 15, 25), 'to': datetime.datetime(2019, 7, 2, 22, 25)}
	city:  {'from': 'Москва', 'to': 'Валенсия'}


crawl_date_time:  2

...

	total_time:  {'minutes': '45', 'hours': '19', 'days': 0}
	date_time:  {'from': datetime.datetime(2019, 7, 2, 18, 45), 'to': datetime.datetime(2019, 7, 3, 13, 30)}
	city:  {'from': 'Москва', 'to': 'Лион'}


crawl_date_time:  2019-04-17 22:29:06.474560
price:  6419
	total_time:  {'minutes': '30', 'hours': '21', 'days': 0}
	date_time:  {'from': datetime.datetime(2019, 7, 2, 18, 45), 'to': datetime.datetime(2019, 7, 3, 15, 15)}
	city:  {'from': 'Москва', 'to': 'Лион'}


crawl_date_time:  2019-04-17 22:29:08.571656
price:  6859
	total_time:  {'minutes': '20', 'hours': '24', 'days': 0}
	date_time:  {'from': datetime.datetime(2019, 7, 2, 18, 45), 'to': datetime.datetime(2019, 7, 3, 18, 5)}
	city:  {'from': 'Москва', 'to': 'Лион'}


crawl_date_time:  2019-04-17 22:29:10.767642
price:  7761
	total_time:  {'minutes': '50', 'hours': '13', 'days': 0}
	date_time:  {'from': datetime.datetime(2019, 7, 2, 8, 45), 'to': datetime.datetime(2019, 7, 2, 21, 35)}
	city:  {'from': 'Москва', 'to': 'Лион'}




	total_time:  {'minutes': '45', 'hours': '2', 'days': '1'}
	date_time:  {'from': datetime.datetime(2019, 7, 2, 17, 40), 'to': datetime.datetime(2019, 7, 3, 19, 25)}
	city:  {'from': 'Москва', 'to': 'Пальма-де-Мальйорка'}


crawl_date_time:  2019-04-17 22:31:39.986746
price:  8508
	total_time:  {'minutes': '55', 'hours': '15', 'days': 0}
	date_time:  {'from': datetime.datetime(2019, 7, 2, 17, 0), 'to': datetime.datetime(2019, 7, 3, 7, 55)}
	city:  {'from': 'Москва', 'to': 'Пальма-де-Мальйорка'}


crawl_date_time:  2019-04-17 22:31:42.294391
price:  8508
	total_time:  {'minutes': '5', 'hours': '4', 'days': '1'}
	date_time:  {'from': datetime.datetime(2019, 7, 2, 17, 0), 'to': datetime.datetime(2019, 7, 3, 20, 5)}
	city:  {'from': 'Москва', 'to': 'Пальма-де-Мальйорка'}


crawl_date_time:  2019-04-17 22:31:45.037833
price:  8652
	total_time:  {'minutes': '55', 'hours': '24', 'days': 0}
	date_time:  {'from': datetime.datetime(2019, 7, 2, 17, 40), 'to': datetime.datetime(2019, 7, 3, 17, 35)}

crawl_date_time:  2019-04-17 22:34:06.068810
price:  6334
	total_time:  {'minutes': 0, 'hours': '14', 'days': 0}
	date_time:  {'from': datetime.datetime(2019, 7, 2, 20, 40), 'to': datetime.datetime(2019, 7, 3, 9, 40)}
	city:  {'from': 'Москва', 'to': 'Милан'}


crawl_date_time:  2019-04-17 22:34:08.261357
price:  6608
	total_time:  {'minutes': '55', 'hours': '24', 'days': 0}
	date_time:  {'from': datetime.datetime(2019, 7, 2, 18, 45), 'to': datetime.datetime(2019, 7, 3, 18, 40)}
	city:  {'from': 'Москва', 'to': 'Милан'}


crawl_date_time:  2019-04-17 22:34:10.722066
price:  6830
	total_time:  {'minutes': '35', 'hours': '3', 'days': 0}
	date_time:  {'from': datetime.datetime(2019, 7, 2, 10, 25), 'to': datetime.datetime(2019, 7, 2, 13, 0)}
	city:  {'from': 'Москва', 'to': 'Милан'}


crawl_date_time:  2019-04-17 22:34:12.604015
price:  7136
	total_time:  {'minutes': '25', 'hours': '8', 'days': 0}
	date_time:  {'from': datetime.datetime(2019, 7, 2, 15, 15), 'to': datetime.datetime(2019, 7,

MOW → NAP 

crawl_date_time:  2019-04-17 22:36:47.145585
price:  8162


KeyboardInterrupt: 

In [35]:
#март
time_list_march = ['0103','0203','0303','0403','0503','0603','0703','0803','0903','1003',
                   '1103','1203','1303','1403','1503','1603','1703','1803','1903','2003',
                   '2103','2203','2303','2403','2503','2603','2703','2803','2903','3003','3103']

#апрель
time_list_april = ['0104','0204','0304','0404','0504','0604','0704','0804','0904','1004',
                   '1104','1204','1304','1404','1504','1604','1704','1804','1904','2004',
                   '2104','2204','2304','2404','2504','2604','2704','2804','2904','3004']

#май
time_list_may = ['0105','0205','0305','0405','0505','0605','0705','0805','0905','1005',
                 '1105','1205','1305','1405','1505','1605','1705','1805','1905','2005',
                 '2105','2205','2305','2405','2505','2605','2705','2805','2905','3005', '3105']

#июнь
time_list_june = ['0106','0206','0306','0406','0506','0606','0706','0806','0906','1006',
                  '1106','1206','1306','1406','1506','1606','1706','1806','1906','2006',
                  '2106','2206','2306','2406','2506','2606','2706','2806','2906','3006']

#июль
time_list_july = ['0107','0207','0307','0407','0507','0607','0707','0807','0907','1007',
                  '1107','1207','1307','1407','1507','1607','1707','1807','1907','2007',
                  '2107','2207','2307','2407','2507','2607','2707','2807','2907','3007', '3107']

#август
time_list_august = ['0108','0208','0308','0408','0508','0608','0708','0808','0908','1008',
                    '1108','1208','1308','1408','1508','1608','1708','1808','1908','2008',
                    '2108','2208','2308','2408','2508','2608','2708','2808','2908','3008', '3108']


city_list = {'BER':'Берлин', 
             'HAM':'Гамбург', 
             'BRE':'Бремен', 
             'HAJ':'Ганновер', 
             'LEJ':'Лейпциг', 
             'DUS':'Дюссельдорф', 
             'CGN':'Кёльн', 
             'NUE':'Нюрнберг', 
             'MUC':'Мюнхен', 
             'PRG':'Прага', 
             'BRQ':'Брно', 
             'KLV':'Карловы Вары', 
             'VIE':'Вена', 
             'SZG':'Зальцбург', 
             'INN':'Инсбрук', 
             'BUD':'Будапешт', 
             'BRU':'Брюссель', 
             'ANR':'Антверпен', 
             'OST':'Брюгге', 
             'NCE':'Ницца', 
             'LYS':'Лион', 
             'PAR':'Париж', 
             'MPL':'Монпелье', 
             'SXB':'Страсбург', 
             'GVA':'Женева', 
             'ZRH':'Цюрих', 
             'BRN':'Берн',
             'MIL':'Милан',
             'ROM':'Рим',
             'VCE':'Венеция',
             'VRN':'Верона',
             'NAP':'Неаполь',
             'TRN':'Турин',
             'FLR':'Флоренция',
             'PMO':'Палермо',
             'BLQ':'Болонья',
             'PEG':'Перуджа',
             'AGP':'Малага',
             'BCN':'Барселона',
             'PMI':'Майорка',
             'MAD':'Мадрид',
             'ALC':'Аликанте',
             'SVQ':'Севилья',
             'VLC':'Валенсия',
             'QCN':'Таррагона',
             'IBZ':'Ибица',
             'LON':'Лондон',
             'MAN':'Манчестер',
             'EDI':'Эдинбург',
             'GLA':'Глазго',
             'CBG':'Кембридж',
             'BFS':'Белфаст',
             'DUB':'Дублин',
             'ORC':'Корк',
             'AMS':'Амстердам',
             'RTM':'Роттердам',
             'EIN':'Эйндховен',
             'LCA':'Ларнака',
             'ECN':'Никосия',
             'PFO':'Пафос',
             'STO':'Стокгольм',
             'GOT':'Гётерберг'
             }

city_list = OrderedDict(sorted(city_list.items(), key=lambda kv: kv[1]))

In [417]:
def printBeautifulTicket(post):
    print('_id: ', post['_id'])
    print('crawl_date_time: ', post['crawl_date_time'])
    print('flights: ')
    for flight in post['flights']:
        print('\tcity: ')
        print('\t\tfrom\t: ', flight['city']['from'])
        print('\t\tto\t: ', flight['city']['to'])
        print('\tdate_time: ')
        print('\t\tfrom\t: ', flight['date_time']['from'])
        print('\t\tto\t: ', flight['date_time']['to'])
        print('\tdetailes: ')
        for period in flight['details']:
            if period['type'] == 'leg':
                print('\t\tairline\t: ', period['airline'])
                print('\t\tairport: ')
                print('\t\t\tfrom: ')
                print('\t\t\t\tiata\t: ', period['airport']['from']['iata'])
                print('\t\t\t\tname\t: ', period['airport']['from']['name'])
                print('\t\t\tto: ')
                print('\t\t\t\tiata\t: ', period['airport']['to']['iata'])
                print('\t\t\t\tname\t: ', period['airport']['to']['name'])
                print('\t\tdate_time: ')
                print('\t\t\tfrom\t: ', period['date_time']['from'])
                print('\t\t\tto\t: ', period['date_time']['to'])
                print('\t\tflight_number\t: ', period['flight_number'])
                print('\t\ttype\t: ', period['type'])
            if period['type'] == 'stop':
                print('\t\tplace\t: ', period['place'])
                print('\t\ttime: ')
                print('\t\t\thours\t: ', period['time']['hours'])
                print('\t\t\tminutes\t: '), period['time']['minutes']
                
    print('\n\n')

In [ ]:
for post in collection.find({'price':61765}):
    printBeautifulTicket(post)

In [ ]:
for post in collection.find({'price':61765}):
    pprint(post)